this script measures the difference between raw research profile and the profile without funding from other countries

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from ast import literal_eval
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import geopandas as gp
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
pubs_path="../../data/nf_folder/Data/DerivedData/CleanedRawData/pubs_author_funder.csv"
kl_output='../../data/nf_folder/Data/DerivedData/Derived/cntry_kl.csv'

In [3]:
pubs_author_funder = pd.read_csv(pubs_path)
pubs_author_funder['author_country'] = pubs_author_funder['author_country'].apply(literal_eval)
pubs_author_funder['funder_country'] = pubs_author_funder['funder_country'].apply(
    lambda x:literal_eval(x) if pd.notnull(x) else "Not-Funded")
pubs_author_funder.head()

/home/miaoli/conda/envs/funding/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,author_country,funder_country,discipline,year,rel_citation,citation
0,30507292,[Turkey],[Turkey],77,2009,0.485,6
1,30574690,"[United States, France]","[United States, United States, United States, ...",83,2009,0.703,9
2,30574691,[Russia],[Russia],83,2009,0.312,4
3,30585095,"[Spain, Spain]","[Spain, Spain, Spain, Spain, Spain, Spain]",89,2009,0.137,1
4,30585097,"[Germany, Spain]",[Spain],89,2009,0.410,3


In [4]:
pubs_author_funder['author_distinct']=pubs_author_funder['author_country'].apply(lambda x:list(set(x)))
pubs_author_funder['funder_distinct']=pubs_author_funder['funder_country'].apply(
    lambda x:list(set(x)) if x!='Not-Funded' else 'Not-Funded')

calculate the research profile of countries using fractionalized method

In [5]:
cntry_frac=[]
for ind,row in pubs_author_funder.iterrows():
    authors=row['author_distinct']
    dis=row['discipline']
    year=row['year']
    for a in authors:
        cntry_frac.append([a,year,dis,1/len(authors)])

In [10]:
test=pd.DataFrame(cntry_frac, columns=['country','year','dis','count'])
test

,country,year,dis,count
0,Turkey,2009,77,1.0
1,France,2009,83,0.5
2,United States,2009,83,0.5
3,Russia,2009,83,1.0
4,Spain,2009,89,1.0
...,...,...,...,...
17371661,Pakistan,2018,89,1.0
17371662,Singapore,2017,44,1.0
17371663,Iran,2017,96,1.0
17371664,Israel,2017,96,1.0


In [11]:
cntry_filter=[]
for ind, row in pubs_author_funder.iterrows():
    authors=row['author_distinct']
    funders=row['funder_distinct']
    dis=row['discipline']
    n_a=len(authors)
    n_f=len(funders)
    year=row['year']
    if funders!='Not-Funded':
        for a in authors:
            if a in funders:
                cntry_filter.append([a,year,dis,1/(n_a*n_f)])
    else:
        for a in authors:
            cntry_filter.append([a,year,dis,1/(n_a)])
    

In [13]:
cntry_filter_df=pd.DataFrame(cntry_filter, columns=['country','year','dis','count'])


,country,dis,count
0,Afghanistan,1,5.027778
1,Afghanistan,2,1.944444
2,Afghanistan,3,1.500000
3,Afghanistan,4,2.000000
4,Afghanistan,5,0.500000
...,...,...,...
14378,Zimbabwe,91,0.500000
14379,Zimbabwe,93,0.500000
14380,Zimbabwe,96,3.083333
14381,Zimbabwe,99,6.333333


In [7]:
dislist=pubs_author_funder.discipline.unique()
dislist.sort()
def get_dis_distribution(resultlist):
    result_df=pd.DataFrame(resultlist,columns=['country','dis','count'])
    result_df=result_df.groupby(['country','dis'])['count'].sum().reset_index(name='count')
    result_df=result_df.pivot_table(index='country',columns='dis',values='count',fill_value=0)
    result_df=result_df[dislist]
    result_df=result_df.div(result_df.sum(axis=1), axis=0)
    return result_df

In [16]:
frac_df=get_dis_distribution(cntry_frac)
frac_df.head()

dis,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.069572,0.027432,0.013881,0.020326,0.002975,0.000000,0.014873,0.000000,0.002975,0.000000,...,0.002975,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001190,0.000000,0.000000
Albania,0.040776,0.035766,0.004527,0.006199,0.002868,0.005111,0.009017,0.009481,0.000626,0.001878,...,0.004381,0.00000,0.003308,0.000626,0.000000,0.009388,0.000626,0.003755,0.001252,0.000000
Algeria,0.026943,0.016177,0.002909,0.006602,0.004900,0.001996,0.004115,0.004279,0.000353,0.001248,...,0.010129,0.00263,0.034616,0.010683,0.014416,0.060955,0.000612,0.010683,0.026440,0.015109
Andorra,0.000000,0.054395,0.000000,0.034355,0.042943,0.000000,0.017177,0.011452,0.011452,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Angola,0.029152,0.047881,0.000757,0.010468,0.012368,0.000000,0.022842,0.038428,0.004152,0.004417,...,0.000000,0.00000,0.001767,0.000000,0.000000,0.000000,0.000000,0.000000,0.001325,0.000000


In [17]:
filter_df=get_dis_distribution(cntry_filter)
filter_df.head()

dis,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.048697,0.018833,0.014528,0.019371,0.004843,0.000000,0.024214,0.000000,0.004843,0.000000,...,0.004843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Albania,0.041038,0.028162,0.003584,0.004325,0.002904,0.005190,0.008705,0.004943,0.000000,0.000000,...,0.005190,0.000000,0.002189,0.000741,0.000000,0.010380,0.000741,0.004449,0.000000,0.000000
Algeria,0.026536,0.015426,0.003024,0.006893,0.004851,0.002011,0.003877,0.004224,0.000297,0.001138,...,0.010771,0.002797,0.035185,0.010327,0.014197,0.060906,0.000539,0.010260,0.026686,0.014503
Andorra,0.000000,0.029244,0.000000,0.029244,0.073109,0.000000,0.029244,0.019496,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Angola,0.036312,0.049569,0.000000,0.006917,0.008646,0.000000,0.019021,0.025505,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
def cal_KL(P,Q):
    sum=0
    for ind,item in enumerate(P):
        value_p=P[ind]
        value_q=Q[ind]
        if value_p!=0 and value_q!=0:
            sum=sum+(value_p*np.log(value_p/value_q))
    return sum

In [19]:
result=[]
for index, row in filter_df.iterrows():
    cntry=index
    p=row.values
    q=frac_df.loc[cntry,:].values
    kl_value=cal_KL(p,q)
    result.append([cntry,kl_value])

In [21]:
result_df=pd.DataFrame(result,columns=['country','kl'])
result_df.head()

,country,kl
0,Afghanistan,0.088699
1,Albania,0.020994
2,Algeria,0.002062
3,Andorra,0.227091
4,Angola,0.115770


In [23]:
result_df.to_csv(kl_output, index=False)